# Reviews

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [138]:
reviews1 = pd.read_csv("reviews_0-250.csv")
reviews2 = pd.read_csv("reviews_250-500.csv")
reviews3 = pd.read_csv("reviews_500-750.csv")
reviews4 = pd.read_csv("reviews_750-1250.csv")
reviews5 = pd.read_csv("reviews_1250-end.csv")
products_final = pd.read_csv("products_final.csv")

C:\Users\bigle\AppData\Local\Temp\ipykernel_27360\4215495468.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews1 = pd.read_csv("reviews_0-250.csv")
C:\Users\bigle\AppData\Local\Temp\ipykernel_27360\4215495468.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews4 = pd.read_csv("reviews_750-1250.csv")
C:\Users\bigle\AppData\Local\Temp\ipykernel_27360\4215495468.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews5 = pd.read_csv("reviews_1250-end.csv")


## Unificamos todas las tablas de reviews

In [140]:
reviews = pd.concat([reviews1,reviews2,reviews3,reviews4,reviews5])
reviews.shape

(1094411, 19)

## Duplicados y nulos

In [142]:
reviews.head(3)

,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0


### Datos duplicados

In [144]:
reviews.duplicated().sum()

0

No tenemos datos duplicados, por lo que no es necesario eliminar ninguna fila.

### Eliminamos las reviews sin valoración y recalculamos helpfulness

In [147]:
reviews = reviews[reviews.loc[:,'total_feedback_count'] != 0]
reviews.loc[:,'helpfulness'] = (reviews.loc[:,'total_pos_feedback_count'] /reviews.loc[:,'total_feedback_count'] * 100).round(2)
reviews.loc[:,'helpfulness'].unique()

array([100.  ,  25.  ,  44.44, ...,  18.29,  26.56,  90.13])

In [148]:
reviews.head()

,Unnamed: 0,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,0,1741593524,5,1.0,100.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
5,5,42802569154,4,1.0,100.0,1,0,1,2023-03-19,The scent isn’t my favourite but it works grea...,Great!,lightMedium,brown,normal,brown,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
6,6,6941883808,2,0.0,25.0,8,6,2,2023-03-19,I’ll give this 2 stars for nice packaging and ...,Dried my lips out and clogged my pores,light,blue,combination,brown,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
8,8,7656791726,5,1.0,100.0,1,0,1,2023-03-18,I love this stuff. I first had the sample size...,Must have.,light,blue,normal,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
9,9,38727834382,5,1.0,100.0,2,0,2,2023-03-18,I purchased the Sweet Candy scent at my local ...,RUN TO PURCHASE,light,brown,normal,brown,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0


In [149]:
reviews.shape

(532819, 19)

Con esto, eliminamos las reviews que no han sido calificadas por otros usuarios. De esta forma nos aseguramos de que las reviews han sido valoradas por otros usuarios.

### Eliminamos las columnas que no son necesarias
Al haber recalculado helpfulness podemos eliminar "total_neg_feedback_count" y "total_pos_feedback_count"

In [152]:
reviews.drop(
    ["total_neg_feedback_count", "total_pos_feedback_count"],
    axis = 1,
    inplace = True
)

Eliminamos 'product_name', 'brand_name' y 'review_title' porque no nos aportarán valor para el modelo

In [155]:
reviews.drop(columns=['product_name', 'review_title'], inplace = True)

In [158]:
reviews.columns

Index(['Unnamed: 0', 'author_id', 'rating', 'is_recommended', 'helpfulness',
       'total_feedback_count', 'submission_time', 'review_text', 'skin_tone',
       'eye_color', 'skin_type', 'hair_color', 'product_id', 'brand_name',
       'price_usd'],
      dtype='object')

Con esto, nos quedamos con las columnas que consideramos importantes.

## Transformación y complementación de los datos

### Cambiamos el nombre de la primera columna a "review_id" y ajustamos los datos

In [180]:
reviews.rename(columns={'Unnamed: 0': 'review_id'}, inplace=True)

In [184]:
reviews['review_id'] = range(1, len(reviews) + 1)

In [188]:
reviews.shape

(532819, 15)

### Nos aseguramos de que 'submission_time' se encuentra en formato fecha

In [191]:
reviews.loc[:,'submission_time'] = pd.to_datetime(reviews.loc[:,'submission_time']).dt.date
reviews['submission_time']

0        2023-02-01
5        2023-03-19
6        2023-03-19
8        2023-03-18
9        2023-03-18
            ...    
49939    2022-07-15
49941    2021-08-24
49942    2023-02-25
49943    2022-10-12
49945    2021-05-26
Name: submission_time, Length: 532819, dtype: object

In [193]:
reviews.shape

(532819, 15)

### Ajustamos los datos de 'eye_color'

In [196]:
reviews.eye_color.unique()

# Reemplazar 'gray' y 'Grey' por 'blue'
reviews.loc[:,'eye_color'] = reviews.loc[:,'eye_color'].replace(['gray', 'Grey'], 'blue')
reviews.eye_color.unique()

array(['brown', 'blue', 'green', 'hazel', nan], dtype=object)

### Agrupamos los datos de 'skin_tone'

In [198]:
skin_tone_new = {
    'porcelain': 'very_light',
    'fair': 'very_light',
    'fairLight': 'very_light',
    'light': 'very_light',
    'lightMedium': 'light_to_medium',
    'medium': 'light_to_medium',
    'olive': 'medium_to_tan',
    'mediumTan': 'medium_to_tan',
    'tan': 'medium_to_tan',
    'rich': 'dark',
    'deep': 'dark',
    'dark': 'dark',
    'ebony': 'dark',
    'notSureST': np.nan
}

reviews.loc[:,'skin_tone'] = reviews.loc[:,'skin_tone'].replace(skin_tone_new)

In [199]:
reviews.skin_tone.unique()

array([nan, 'light_to_medium', 'very_light', 'medium_to_tan', 'dark'],
      dtype=object)

### Reemplazamos los datos de 'hair_color'

In [201]:
reviews.loc[:,'hair_color'] = reviews.loc[:,'hair_color'].replace({
    'brunette': 'brown',
    'auburn': 'red'
})

In [202]:
reviews.hair_color.unique()

array(['black', 'brown', 'blonde', nan, 'red', 'gray'], dtype=object)

### Creamos la columna 'is_recomended' basada en la columna 'rating'

Con esta columna consideraremos como recomendación las reviews con ratings de 4 o 5

In [205]:
reviews.loc[:,'is_recommended'] = (reviews['rating'] > 3).astype(int)

In [206]:
reviews.head()

,review_id,author_id,rating,is_recommended,helpfulness,total_feedback_count,submission_time,review_text,skin_tone,eye_color,skin_type,hair_color,product_id,brand_name,price_usd
0,1,1741593524,5,1.0,100.0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,NaN,brown,dry,black,P504322,NUDESTIX,19.0
5,2,42802569154,4,1.0,100.0,1,2023-03-19,The scent isn’t my favourite but it works grea...,light_to_medium,brown,normal,brown,P420652,LANEIGE,24.0
6,3,6941883808,2,0.0,25.0,8,2023-03-19,I’ll give this 2 stars for nice packaging and ...,very_light,blue,combination,brown,P420652,LANEIGE,24.0
8,4,7656791726,5,1.0,100.0,1,2023-03-18,I love this stuff. I first had the sample size...,very_light,blue,normal,blonde,P420652,LANEIGE,24.0
9,5,38727834382,5,1.0,100.0,2,2023-03-18,I purchased the Sweet Candy scent at my local ...,very_light,brown,normal,brown,P420652,LANEIGE,24.0


### Rellenamos los datos faltantes

In [208]:
reviews.isnull().sum()

review_id                    0
author_id                    0
rating                       0
is_recommended               0
helpfulness                  0
total_feedback_count         0
submission_time              0
review_text                447
skin_tone                88522
eye_color               103605
skin_type                56265
hair_color              108843
product_id                   0
brand_name                   0
price_usd                    0
dtype: int64

In [209]:
# Calculamos la moda para cada product_id y cada columna
moda_por_producto = reviews.groupby('product_id').agg({
    'hair_color': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'eye_color': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'skin_tone': lambda x: x.mode().iloc[0] if not x.mode().empty else None,
    'skin_type': lambda x: x.mode().iloc[0] if not x.mode().empty else None
}).reset_index()

In [210]:
# Combina reviews1 con las modas calculadas
reviews = reviews.merge(moda_por_producto, on='product_id', suffixes=('', '_moda'))

# Rellena los valores nulos en cada columna usando las columnas de moda
for column in ['hair_color', 'eye_color', 'skin_tone', 'skin_type']:
    reviews[column].fillna(reviews[f'{column}_moda'], inplace=True)

# Elimina las columnas adicionales
reviews.drop(columns=[f'{column}_moda' for column in ['hair_color', 'eye_color', 'skin_tone', 'skin_type']], inplace=True)

C:\Users\bigle\AppData\Local\Temp\ipykernel_27360\793006523.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reviews[column].fillna(reviews[f'{column}_moda'], inplace=True)


In [214]:
reviews.isnull().sum()

review_id                 0
author_id                 0
rating                    0
is_recommended            0
helpfulness               0
total_feedback_count      0
submission_time           0
review_text             447
skin_tone                 6
eye_color                 2
skin_type                 2
hair_color               19
product_id                0
brand_name                0
price_usd                 0
dtype: int64

### Eliminamos las filas que contienen nulos

In [217]:
reviews.dropna(inplace = True)

In [218]:
reviews.isnull().sum()

review_id               0
author_id               0
rating                  0
is_recommended          0
helpfulness             0
total_feedback_count    0
submission_time         0
review_text             0
skin_tone               0
eye_color               0
skin_type               0
hair_color              0
product_id              0
brand_name              0
price_usd               0
dtype: int64

In [219]:
reviews.head()

,review_id,author_id,rating,is_recommended,helpfulness,total_feedback_count,submission_time,review_text,skin_tone,eye_color,skin_type,hair_color,product_id,brand_name,price_usd
1,2,42802569154,4,1.0,100.00,1,2023-03-19,The scent isn’t my favourite but it works grea...,light_to_medium,brown,normal,brown,P420652,LANEIGE,24.0
2,3,6941883808,2,0.0,25.00,8,2023-03-19,I’ll give this 2 stars for nice packaging and ...,very_light,blue,combination,brown,P420652,LANEIGE,24.0
3,4,7656791726,5,1.0,100.00,1,2023-03-18,I love this stuff. I first had the sample size...,very_light,blue,normal,blonde,P420652,LANEIGE,24.0
4,5,38727834382,5,1.0,100.00,2,2023-03-18,I purchased the Sweet Candy scent at my local ...,very_light,brown,normal,brown,P420652,LANEIGE,24.0
5,6,22249699172,1,0.0,44.44,9,2023-03-17,Honestly I was so excited when I got this in t...,very_light,brown,dry,brown,P420652,LANEIGE,24.0


## Filtrado de usuarios

In [224]:
reviews.shape

(532351, 15)

In [225]:
reviews.author_id.nunique()

329984

Tenenemos un total de 329984, muchos de ellos con una única review de productos. Esto nos complica generar perfiles claros de clientes.

### Renombramos los ID de clientes mal guardados y los hacemos numéricos

In [229]:
# Función para eliminar las letras y conservar solo los números
def extraer_numeros(author_id):
    # Convertir a cadena de texto para asegurarnos de que podemos aplicar la expresión regular
    author_id = str(author_id)
    return re.sub(r'\D', '', author_id)  # \D significa cualquier carácter que no sea un dígito

# Aplicar la función a la columna 'author_id' para limpiar los datos
reviews['author_id'] = reviews['author_id'].apply(extraer_numeros)

In [230]:
reviews.shape

(532351, 15)

### Nos quedamos con usuarios con 3 o más reviews

In [232]:
z= reviews.groupby("author_id").size().sort_values(ascending= False)
df_author_reviews = z.reset_index(name='review_count')

# Renombrar las columnas para mayor claridad
df_author_reviews.rename(columns={'author_id': 'author_id', 0: 'review_count'}, inplace=True)

In [233]:
df_author_reviews

,author_id,review_count
0,1696370280,245
1,1288462295,148
2,1930716686,100
3,24998050674,89
4,25452917574,80
...,...,...
296837,31404687006,1
296838,31404986546,1
296839,31404992779,1
296840,31405240081,1


In [234]:
reviews.shape

(532351, 15)

In [235]:
authors_final = df_author_reviews[df_author_reviews["review_count"]>=3]
authors_final

,author_id,review_count
0,1696370280,245
1,1288462295,148
2,1930716686,100
3,24998050674,89
4,25452917574,80
...,...,...
44489,5879202968,3
44490,6033307138,3
44491,969758111,3
44492,12783103399,3


usuarios unicos = 329984
usuarios con mas de 1 review = 101352

### Filtramos la dataframe para quedarnos solo con las reviews de los usuarios correctos

In [238]:
reviews_final = reviews[reviews['author_id'].isin(authors_final['author_id'])]

In [240]:
reviews_final.shape

(219106, 15)

### Columna de número de review
Generamos una columna donde se muestre cual es el número de cada review generada por cada usuario.

In [242]:
reviews_final.loc[:,'numero_review'] = reviews_final.groupby('author_id').cumcount() + 1

C:\Users\bigle\AppData\Local\Temp\ipykernel_27360\3112920716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_final.loc[:,'numero_review'] = reviews_final.groupby('author_id').cumcount() + 1


In [243]:
reviews_final.head(2)

,review_id,author_id,rating,is_recommended,helpfulness,total_feedback_count,submission_time,review_text,skin_tone,eye_color,skin_type,hair_color,product_id,brand_name,price_usd,numero_review
1,2,42802569154,4,1.0,100.0,1,2023-03-19,The scent isn’t my favourite but it works grea...,light_to_medium,brown,normal,brown,P420652,LANEIGE,24.0,1
2,3,6941883808,2,0.0,25.0,8,2023-03-19,I’ll give this 2 stars for nice packaging and ...,very_light,blue,combination,brown,P420652,LANEIGE,24.0,1


## Clusterización de usuarios

### Creamos una columna de cluster de productos
Traemos del dataframe de productos la columna de cluster de producto para así poder ver que tipo de productos compra cada usuario.

In [246]:
reviews_final = reviews_final.merge(
    products_final[['product_id', 'cluster_product']],
    on='product_id',
    how='left'
)

In [247]:
reviews_final.head(2)

,review_id,author_id,rating,is_recommended,helpfulness,total_feedback_count,submission_time,review_text,skin_tone,eye_color,skin_type,hair_color,product_id,brand_name,price_usd,numero_review,cluster_product
0,2,42802569154,4,1.0,100.0,1,2023-03-19,The scent isn’t my favourite but it works grea...,light_to_medium,brown,normal,brown,P420652,LANEIGE,24.0,1,3
1,3,6941883808,2,0.0,25.0,8,2023-03-19,I’ll give this 2 stars for nice packaging and ...,very_light,blue,combination,brown,P420652,LANEIGE,24.0,1,3


## Guardamos el CSV

In [257]:
reviews_final.shape

(219106, 17)

In [273]:
reviews_final.to_csv('reviews_final.csv', index=False)

In [274]:
len(reviews.product_id.unique())

2329